In [12]:
import pandas as pd
import regex as re

In [13]:
# Import AOP EC table data
AOP_EC_table = pd.read_csv("../aop_ke_ec.csv")

#Set output file name
outfile = "../aop150_from_script.ttl"


In [14]:
AOP150 = AOP_EC_table[AOP_EC_table["AOP"] == "Aop:150"]


In [15]:
# write header 
header = '''@prefix : <http://www.semanticweb.org/mmandal/ontologies/2022/4/untitled-ontology-76#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@base <http://www.w3.org/2002/07/owl#> .

[ rdf:type owl:Ontology ;
   owl:imports <http://purl.obolibrary.org/obo/go/releases/2022-01-13/go.owl> ,
               <http://purl.obolibrary.org/obo/ro/releases/2022-01-20/ro.owl>
 ] .
'''

with open(outfile, "w+") as f:
    f.write(header)

In [16]:
def process_term(input_id, name):
    """
    Takes a term and id and returns ttl class and instance statements
    """
    if ":" in input_id:
        input_id_str = f"{input_id[:2]}_{input_id[3:]}"
    else:
        input_id_str = input_id
    name = re.sub(" ", "_", name)
    class_statement = f'''\n###  http://purl.obolibrary.org/obo/{input_id_str}\n\t<http://purl.obolibrary.org/obo/{input_id_str}> rdf:type owl:Class .\n\n'''
#         with open(outfile, "a") as f:
#             f.write(class_statement)
    instance_statement = f'''###  http://www.co-ode.org/ontologies/ont.owl#{name}\n<http://www.co-ode.org/ontologies/ont.owl#{name}> rdf:type owl:NamedIndividual ,\n\t<http://purl.obolibrary.org/obo/{input_id_str}> '''
#         with open(outfile, "a") as f:
#             f.write(instance_statement)
    return class_statement, instance_statement;

def get_action_id(action):
    if action == "increased":
        return "RO_0000057"
    elif action == "decreased":
        return "RO_0000058"
    else:
        return ""

def get_action_id_steps(step1_ao, step1_source, step2_ao, step2_source):
    if step1_source == "GO" and step2_source == "GO":
        if action == "reduced":
        return "RO_0000057":
    elif action == "decreased":
        return "RO_0000058"
    else:
        return ""
    
def add_action(action, obj_name, proc_phen_name):
    action_id = get_action_id(action)
    proc_phen_name = re.sub(" ", "_", proc_phen_name)
    if action_id != "":
        action_statement = f";\n\t<http://purl.obolibrary.org/obo/{action_id}> <http://www.co-ode.org/ontologies/ont.owl#{proc_phen_name}> "
        return(action_statement)
    else:
        return("")

IndentationError: expected an indented block (3749909078.py, line 26)

In [ ]:
classes = {}
instances = {}
steps = {}
for index, row in AOP150.iterrows():
    row = row.rename(lambda x: re.sub("[\s\/]", "_", x.lower()))
    if row.object_id not in classes.keys() and not pd.isna(row.object_id):
        class_statement, instance_statement = process_term(row.object_id, row.object_term)
        classes[row.object_id] = class_statement
        instances[row.object_id] = instance_statement
    if row.process_phenotype_id not in instances.keys() and not pd.isna(row.process_phenotype_id):
        class_statement, instance_statement = process_term(row.process_phenotype_id, row.process_phenotype_term)
        classes[row.process_phenotype_id] = class_statement
        instances[row.process_phenotype_id] = instance_statement
    action_statement = add_action(row.action,row.object_term, row.process_phenotype_id)
    if not pd.isna(row.object_id):
        instances[row.object_id] += action_statement
    step = int(row.step)
    if step in steps.keys():
        steps[step] = steps[step] + [(row.adverse_outcome, row.process_phenotype_source, row.process_phenotype_id)]
    else:
        steps[step] = [(row.adverse_outcome, row.process_phenotype_source, row.process_phenotype_id)]
for step in range(2,max(steps.keys())+1):
    for step1_item in list(set(steps[step-1])):
        for step2_item in list(set(steps[step])):
            step1_ao, step1_source, pp_id_1 = step1_item
            step2_ao, step2_source, pp_id_2 = step2_item
            action_statement = add_action(step1_ao, step1_source, step2_ao, step2_source)
            
for c, s in classes.items():
    with open(outfile, "a") as f:
            f.write(s)
for i, s in instances.items():
    with open(outfile, "a") as f:
            f.write(s + ".\n\n")
